In [ ]:
import torch
cuda=torch.cuda.is_available()
print(cuda)

In [ ]:
import os
import torch
import torch
import tarfile
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split

In [ ]:
# Dowload the dataset
dataset_url = "https://s3.amazonaws.com/fast-ai-imageclas/cifar10.tgz"
download_url(dataset_url, '.')

In [ ]:
with tarfile.open('./cifar10.tgz', 'r:gz') as tar:
    tar.extractall(path='./data')

In [ ]:
data_dir = './data/cifar10'

print(os.listdir(data_dir))
classes = os.listdir(data_dir + "/train")
print(classes)

In [ ]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor

In [ ]:
dataset = ImageFolder(data_dir+'/train',transform=ToTensor())

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

matplotlib.rcParams['figure.facecolor']='#ffffff'

In [ ]:
def show_example(img,label):
  print('Label :',dataset.classes[label],"("+str(label)+")")
  plt.imshow(img.permute(1,2,0))

In [ ]:
show_example(*dataset[0])

In [ ]:
random_seed=42
torch.manual_seed(random_seed);
val_size=5000
train_size=len(dataset)-val_size
train_ds,val_ds=random_split(dataset,[train_size,val_size])

In [ ]:
from torch.utils.data.dataloader import DataLoader
batchsize=128

In [ ]:
training_dl=DataLoader(train_ds,batchsize,shuffle=True,num_workers=2,pin_memory=True)
val_dl=DataLoader(val_ds,batchsize*2,num_workers=2,pin_memory=True)

In [ ]:
from torchvision.utils import make_grid
def show_data(dl):
  for images,labels in dl:
    fig,ax=plt.subplots(figsize=(12,6))
    ax.set_xticks([]);ax.set_yticks([])
    ax.imshow(make_grid(images,nrow=16).permute(1,2,0))
    break

In [ ]:
show_data(training_dl)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class ImageClassificationBase(nn.Module):
  def training_step(self,batch):
    images,labels=batch
    out=self(images)
    loss=F.cross_entropy(out,labels)
    return loss
  def validation_step(self,batch):
    images,labels=batch
    out=self(images)
    loss=F.cross_entropy(out,labels)
    acc=accuracy(out,labels)
    return{'val_loss':loss.detach(),'val_acc':acc}
  def validation_epoch_end(self,outputs):
    batch_losses=[x['val_loss'] for x in outputs]
    epoch_loss=torch.stack(batch_losses).mean()
    batch_acc=[x['val_acc'] for x in outputs]
    epoch_acc=torch.stack(batch_acc).mean()
    return{'val_loss':epoch_loss.item(),'val_acc':epoch_acc.item()}
  def epoch_end(self,epoch,result):
    print("Epoch[{}],train_loss:{:.4f},val_loss:{:.4f},val_acc:{:.4f}".format(epoch,result['train_loss'],result['val_loss'],result['val_acc']))
def accuracy(outputs,labels):
  _,preds=torch.max(outputs,dim=1)
  return torch.tensor(torch.sum(preds==labels).item()/len(preds))


In [ ]:
import torchvision.models as model
class cifar10cnnModel(ImageClassificationBase):
  def __init__(self):
    super().__init__()
    self.model = model.resnet50(pretrained=True)
    num_ftrs = self.model.fc.in_features
    self.model.fc = nn.Linear(num_ftrs, len(dataset.classes))

  def forward(self,xb):
    return self.model(xb)

In [ ]:
model = cifar10cnnModel()
model

In [ ]:
for images, labels in training_dl:
  print('images.shape:', images.shape)
  out = model(images)
  print('out.shape:',out.shape)
  print('out[0]:' , out[0])
  break

In [ ]:
def get_default_device():
  if torch.cuda.is_available():
    return torch.device('cuda')
  else:
    return torch.device('cpu')
def to_device(data,device):
  if isinstance(data,(list,tuple)):
    return [to_device(x,device) for x in data]
  return data.to(device,non_blocking=True)

class DeviceDataLoader():
  def __init__(self,dl,device):
    self.dl=dl
    self.device=device

  def __iter__(self):
    for b in self.dl:
      yield to_device(b,self.device)

  def __len__(self):
    return len(self.dl)

In [ ]:
device=get_default_device()
device

In [ ]:
train_dl = DeviceDataLoader(training_dl,device)
val_dl = DeviceDataLoader(val_dl,device)
to_device(model, device);

In [ ]:
@torch.no_grad()
def evaluate(model,val_loader):
  model.eval()
  outputs=[model.validation_step(batch) for batch in val_loader]
  return model.validation_epoch_end(outputs)
def fit(epochs,lr,model,train_loader,val_loader,opt_func=torch.optim.SGD):
  history = []
  optimizer  = opt_func(model.parameters(), lr)
  for epoch in range(epochs):
    #training phase
    model. train()
    train_losses = []
    for batch in train_loader:
      loss = model. training_step(batch)
      train_losses. append(loss)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
    result = evaluate (model, val_loader)
    result['train_loss'] = torch.stack(train_losses).mean().item()
    model. epoch_end(epoch, result)
    history.append(result)
  return history

In [ ]:
model = to_device(cifar10cnnModel(), device)

In [ ]:
evaluate(model,val_dl)

In [ ]:
num_epochs = 10
opt_func = torch.optim.SGD
lr = 0.0001

In [ ]:
history=fit(num_epochs, lr, model, train_dl, val_dl, opt_func)

In [ ]:
test_dataset = ImageFolder(data_dir+'/test',transform=ToTensor())

In [ ]:
def predict_image(img, model):
    xb = to_device(img. unsqueeze(0), device)
    yb = model(xb)
    _,preds = torch.max(yb, dim=1)
    return dataset. classes [preds[0].item()]

In [ ]:
img , label = test_dataset[123]
plt.imshow(img.permute(1,2,0))
print('Label :', dataset.classes[label], ',Predicted :', predict_image(img,model))

In [ ]:
img , label = test_dataset[5614]
plt.imshow(img.permute(1,2,0))
print('Label :', dataset.classes[label], ',Predicted :', predict_image(img,model))

In [ ]:
dataset.classes